In [47]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from keras.preprocessing.image import ImageDataGenerator

from keras.utils import np_utils
from scipy import io
import numpy as np
from sklearn.cross_validation import train_test_split
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint

import seaborn as sns
sns.set_style("white")

In [54]:
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
train_data_generator = train_datagen.flow_from_directory('dane/data_train',
                                            target_size=(150, 150),
                                            batch_size=32,
                                            class_mode='binary',
                                            classes=['non-trypophobic','trypophobic'])

test_datagen = ImageDataGenerator(rescale=1./255)
test_data_generator = test_datagen.flow_from_directory('dane/data_valid',
                                            target_size=(150, 150),
                                            batch_size=32,
                                            class_mode='binary',
                                            classes=['non-trypophobic','trypophobic'])

Found 1560 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


In [61]:
model = Sequential()
model.add(Convolution2D(32, 3, 3, border_mode='valid', input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(64, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Convolution2D(128, 3, 3, border_mode='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dropout(0.5))
# model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='softmax'))
# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [62]:
from keras_sequential_ascii import sequential_model_to_ascii_printout
sequential_model_to_ascii_printout(model)

      OPERATION           DATA DIMENSIONS   WEIGHTS(N)   WEIGHTS(%)

          Input   #####   (150, 150, 3)
  Convolution2D    \|/  -------------------       896     0.7%
           relu   #####   (148, 148, 32)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (74, 74, 32)
  Convolution2D    \|/  -------------------     18496    14.2%
           relu   #####   (72, 72, 64)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (36, 36, 64)
  Convolution2D    \|/  -------------------     73856    56.7%
           relu   #####   (34, 34, 128)
   MaxPooling2D   YYYYY -------------------         0     0.0%
                  #####   (17, 17, 128)
        Flatten   ||||| -------------------         0     0.0%
                  #####   (36992,)
        Dropout    | || -------------------         0     0.0%
                  #####   (36992,)
          Dense   XXXXX -------------------     36993    28.4%
        softmax 

In [44]:
from keras_tqdm import TQDMNotebookCallback

In [63]:
history = model.fit_generator(
        train_data_generator,
        samples_per_epoch=2000,
        nb_epoch=50,
        validation_data=test_data_generator,
        nb_val_samples=800)
  #      verbose=0,
  #      callbacks=[TQDMNotebookCallback(leave_inner=False)])

Epoch 1/50


InvalidArgumentError: Received a label value of 1 which is outside the valid range of [0, 1).  Label values: 1 0 1 0 1 1 0 1 1 0 0 1 0 1 1 0 0 0 0 0 1 1 1 1 1 0 0 0 1 0 0 0
	 [[Node: SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits = SparseSoftmaxCrossEntropyWithLogits[T=DT_FLOAT, Tlabels=DT_INT64, _device="/job:localhost/replica:0/task:0/cpu:0"](Reshape_63, Cast_26)]]

Caused by op 'SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.5/dist-packages/ipykernel/kernelapp.py", 